In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns


import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [2]:
dataset = pd.read_csv("Spotify Million Song Dataset_exported.csv")

In [3]:
dataset

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \nLet the angels fly l...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \nMore power \nPower to...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \nis something i'll believe \nf...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \nam i frightened \nwhere can ...


In [4]:
small_dataset = dataset.head(1000)


In [5]:
small_dataset.head()


,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [6]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

C:\Users\himan\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
C:\Users\himan\anaconda3\Lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [7]:
lyrics = small_dataset['text']


In [8]:
lyrics

0      Look at her face, it's a wonderful face  \nAnd...
1      Take it easy with me, please  \nTouch me gentl...
2      I'll never know why I had to go  \nWhy I had t...
3      Making somebody happy is a question of give an...
4      Making somebody happy is a question of give an...
                             ...                        
995    Here I am again  \nWaiting on the moment you d...
996    Oh  \nOh  \nOh oh oh yeah  \nOh no  \nYeah  \n...
997    [Chorus]  \nYou're my shining star  \nThat is ...
998    Ignorance of people purchasing diamonds and ne...
999    When you come back I won't be here  \nShe said...
Name: text, Length: 1000, dtype: object

In [9]:
def clean_text(text, max_length=578):
    cleaned_text = ' '.join(text.split())
    cleaned_text = cleaned_text.lower()
    cleaned_text = cleaned_text.translate(str.maketrans("", "", string.punctuation))
    cleaned_text = ''.join([i for i in cleaned_text if not i.isdigit()])
    cleaned_text = ''.join([i for i in cleaned_text if i.isalpha() or i.isspace()])
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(cleaned_text)
    cleaned_text = ' '.join([word for word in word_tokens if word.lower() not in stop_words])
    
    padded_text = cleaned_text.ljust(max_length)
    
    return padded_text

cleaned_lyrics = lyrics.apply(clean_text)

In [10]:
cleaned_lyrics


0      look face wonderful face means something speci...
1      take easy please touch gently like summer even...
2      ill never know go put lousy rotten show boy to...
3      making somebody happy question give take learn...
4      making somebody happy question give take learn...
                             ...                        
995    waiting moment decide leave stranded edge nowh...
996    oh oh oh oh oh yeah oh yeah shorty aint got ca...
997    chorus youre shining star one like baby angels...
998    ignorance people purchasing diamonds necklaces...
999    come back wont said gently pulled near want ta...
Name: text, Length: 1000, dtype: object

In [11]:
# loop through each song lyric and assign it a label

emotion = []

for i in range(len(cleaned_lyrics)):
    emt = classifier(cleaned_lyrics[i])
    max_label = max(emt[0], key=lambda x: x['score'])['label']
    emotion.append(max_label)

In [17]:
emotion

['joy',
 'joy',
 'sadness',
 'joy',
 'joy',
 'sadness',
 'sadness',
 'sadness',
 'surprise',
 'sadness',
 'sadness',
 'joy',
 'sadness',
 'joy',
 'joy',
 'joy',
 'surprise',
 'sadness',
 'fear',
 'neutral',
 'sadness',
 'sadness',
 'sadness',
 'sadness',
 'joy',
 'sadness',
 'joy',
 'sadness',
 'sadness',
 'fear',
 'sadness',
 'joy',
 'sadness',
 'joy',
 'surprise',
 'fear',
 'sadness',
 'joy',
 'fear',
 'sadness',
 'fear',
 'fear',
 'sadness',
 'sadness',
 'surprise',
 'sadness',
 'sadness',
 'fear',
 'fear',
 'sadness',
 'anger',
 'sadness',
 'fear',
 'joy',
 'sadness',
 'joy',
 'sadness',
 'anger',
 'joy',
 'sadness',
 'sadness',
 'sadness',
 'sadness',
 'fear',
 'joy',
 'sadness',
 'fear',
 'fear',
 'fear',
 'sadness',
 'joy',
 'joy',
 'joy',
 'anger',
 'joy',
 'joy',
 'anger',
 'fear',
 'sadness',
 'sadness',
 'sadness',
 'joy',
 'anger',
 'sadness',
 'sadness',
 'fear',
 'sadness',
 'joy',
 'joy',
 'joy',
 'sadness',
 'joy',
 'sadness',
 'surprise',
 'sadness',
 'joy',
 'fear',
 

In [12]:
small_dataset['Emotion']=emotion

C:\Users\himan\AppData\Local\Temp\ipykernel_6272\1247694247.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_dataset['Emotion']=emotion


In [13]:
small_dataset.to_csv('spotify_song_emotions.csv', index=False)


In [14]:
df = pd.read_csv("spotify_song_emotions.csv")


In [15]:
df

,artist,song,link,text,Emotion
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd...",joy
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...",joy
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,sadness
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,joy
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,joy
...,...,...,...,...,...
995,Backstreet Boys,Satellite,/b/backstreet+boys/satellite_20733842.html,Here I am again \nWaiting on the moment you d...,sadness
996,Backstreet Boys,She's A Dream,/b/backstreet+boys/shes+a+dream_20794158.html,Oh \nOh \nOh oh oh yeah \nOh no \nYeah \n...,surprise
997,Backstreet Boys,Shining Star,/b/backstreet+boys/shining+star_20011554.html,[Chorus] \nYou're my shining star \nThat is ...,joy
998,Backstreet Boys,Shout,/b/backstreet+boys/shout_20595461.html,Ignorance of people purchasing diamonds and ne...,anger


In [16]:
df.shape


(1000, 5)